In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
january_df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [3]:
total_samples = january_df.shape[0]
january_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
january_df['duration'] = (january_df['tpep_dropoff_datetime'] - january_df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [5]:
january_df['duration'].std()

42.59435124195458

In [6]:
january_df = january_df[(january_df['duration'] >= 1) & (january_df['duration'] <= 60)]

In [7]:
january_df.shape[0] / total_samples ## total after cleaning

0.9812202822125979

In [8]:
features_to_use = ['PULocationID', 'DOLocationID']
january_df[features_to_use] = january_df[features_to_use].astype(str)
records = january_df[features_to_use].to_dict(orient='records')

In [9]:
vectorizer = DictVectorizer()
X_train = vectorizer.fit_transform(records)
print('Columns in matrix: ', X_train.shape[1])
y_train = january_df['duration'].values

Columns in matrix:  515


In [10]:
del records
del january_df

In [11]:
lr = LinearRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [12]:
root_mean_squared_error(y_train, y_pred)


7.649261931816197

In [13]:
del y_train
del y_pred
del X_train

In [17]:
february_df = pd.read_parquet('data/yellow_tripdata_2023-02.parquet')
february_df['duration'] = (february_df['tpep_dropoff_datetime'] - february_df['tpep_pickup_datetime']).dt.total_seconds() / 60
february_df = february_df[(february_df['duration'] >= 1) & (february_df['duration'] <= 60)]
february_df[features_to_use] = february_df[features_to_use].astype(str)
records = february_df[features_to_use].to_dict(orient='records')

X_test = vectorizer.transform(records)
y_test = february_df['duration'].values

In [18]:
y_pred = lr.predict(X_test)

In [19]:
root_mean_squared_error(y_test, y_pred)

7.8118186871593